<a href="https://colab.research.google.com/github/orus-kade/pytorch_fatigue_classification/blob/master/demo_network_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
sys.path.append('/content/drive/My Drive/fatigue')
import transforms as T
from torchvision import transforms, models
import matplotlib.pyplot as plt
from PIL import Image



In [52]:
# dir_path = '/content/drive/My Drive/fatigue/faces/no_dataset_faces'
dir_path = '/content/drive/My Drive/fatigue/faces/val_data_second_minute/0_normal/08'
target = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#For CNN

In [72]:
def load_model_CNN(model_name, model_path, device):
  if model_name == 'r3d_18':
    model = models.video.r3d_18(pretrained=False)
  elif model_name == 'r2plus1d_18':
    model = models.video.r2plus1d_18(pretrained=False)
  elif model_name == 'mc3_18':
    model = models.video.mc3_18(pretrained=False)
  else:
    return None  
  model.fc = torch.nn.Linear(model.fc.in_features, 2)  
  model = torch.load(model_path)
  model.eval()
  model = model.to(device)
  return model

def load_data_for_CNN_from_dir(dir_path):
  paths = os.listdir(dir_path)
  paths.sort()
  my_face = paths[:15]
  transforms1 = transforms.Compose([transforms.Resize((112,112)), transforms.ToTensor()])
  transforms2 = transforms.Compose([                                      
            T.ToFloatTensorInZeroOne(),
            T.Normalize(mean = [0.43216, 0.394666, 0.37645], std = [0.22803, 0.22145, 0.216989])
        ])
  clip = torch.Tensor()
  for img_file in [dir_path+os.path.sep+x for x in my_face]:
    img = Image.open(img_file)
    img_tensor = transforms1(img)
    clip = torch.cat((clip, img_tensor.unsqueeze(0)), dim = 0)
  clip = clip.permute(0,2,3,1)
  clip = transforms2(clip)  
  return clip

def load_data_for_CNN_form_img_arr(img_arr):
  transforms1 = transforms.Compose([transforms.Resize((112,112)), transforms.ToTensor()])
  transforms2 = transforms.Compose([                                      
            T.ToFloatTensorInZeroOne(),
            T.Normalize(mean = [0.43216, 0.394666, 0.37645], std = [0.22803, 0.22145, 0.216989])
        ])
  clip = torch.Tensor()
  for img in img_arr:
    img_tensor = transforms1(img)
    clip = torch.cat((clip, img_tensor.unsqueeze(0)), dim = 0)
  clip = clip.permute(0,2,3,1)
  clip = transforms2(clip)  
  return clip

def show_clip_frames_CNN(clip):
  mean = np.array([0.43216, 0.394666, 0.37645])
  std = np.array([0.22803, 0.22145, 0.216989])
  fgs, axis = plt.subplots(3,5, figsize = (20,10))
  for ax, i in zip (axis.ravel(), range(clip.shape[1])):
    frame = clip.permute(1, 0, 2, 3)[i]
    frame = (frame.permute(1,2,0).numpy() * std + mean) * 255
    ax.imshow(frame)  

def predict_CNN(model, clip, device):
  with torch.set_grad_enabled(False):
    pred = model(torch.unsqueeze(clip, dim=0).to(device))
    pred = pred.argmax(dim=1).to('cpu').numpy()
    return pred

# For CNN+RNN

#Test CNN

In [43]:
# %%time

# ResNet(2+1)D все слои

model_name = 'r2plus1d_18'
model_path = '/content/drive/My Drive/fatigue/results/4rd mc3_18 pretrained/model_r2plus1d_18_all.pth'

model = load_model_CNN(model_name, model_path, device)   
# print(model)
clip_data = load_data_for_CNN(dir_path)
show_clip_frames_CNN(clip_data)
print('Target {}'.format(target))
pred = predict_CNN(model, clip_data, device)
print('predict {}'.format(pred))


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv3d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Target 0
predict [0]
CPU times: user 645 ms, sys: 125 ms, total: 770 ms
Wall time: 862 ms


## Some time tests

In [42]:
%%time
model_name = 'r2plus1d_18'
model_path = '/content/drive/My Drive/fatigue/results/4rd mc3_18 pretrained/model_r2plus1d_18_all.pth'
model = load_model_CNN(model_name, model_path, device)

CPU times: user 564 ms, sys: 80.6 ms, total: 644 ms
Wall time: 727 ms


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv3d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [54]:
%%time
clip_data = load_data_for_CNN(dir_path)

Время :
0.05176234245300293


In [67]:
img_arr = []
paths = os.listdir(dir_path)
paths.sort()
my_face = paths[:15]
for img_file in [dir_path+os.path.sep+x for x in my_face]:
  img = Image.open(img_file)
  img_arr.append(img)

In [71]:
%%time
clip = load_data_for_CNN_form_img_arr(img_arr)

CPU times: user 15.4 ms, sys: 907 µs, total: 16.3 ms
Wall time: 17.2 ms


In [59]:
%%time
print('Target {}'.format(target))
pred = predict_CNN(model, clip_data, device)
print('predict {}'.format(pred))

Target 0
predict [0]
Время :
0.09074735641479492
CPU times: user 54.7 ms, sys: 33 ms, total: 87.7 ms
Wall time: 90.8 ms


#Test CNN+RNN